In [1]:
import pysam
import os
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
#check error
script = "/home/wenjingu/remillsscr/MCV_fusion/searcHPV_script1/"

In [3]:
!ls /home/wenjingu/remillsscr/MCV_fusion/searcHPV_script1/

run.sh	       Sample_126469  Sample_126497  Sample_126511
Sample_126457  Sample_126489  Sample_126499  Sample_126515
Sample_126461  Sample_126491  Sample_126501  Sample_126517
Sample_126467  Sample_126495  Sample_126505  Sample_126521


In [4]:
samples = os.listdir(script)
for sample in samples:
    if "Sample" in sample:
        with open(f'{script}/{sample}/{sample}_searcMCV.err') as errFile:
            if "error" in errFile.read() or "Error" in errFile.read():
                print(sample)

In [5]:
#read dataframe
MCVinfo = pd.read_excel('MCC_MCPV_v1.xlsx')

In [6]:
############integration fell into genome###########

In [7]:
!ls /home/wenjingu/remillsscr/HPV_fusion/ref_hg19/

align_firstTwo.py				 HPV18.fa.pac
annotations					 HPV18.fa.sa
BuildInfo.current.txt				 HPV18.nhr
CCDS.current.txt				 HPV18.nin
chrLength.txt					 HPV18.nsq
chrNameLength.txt				 HPV18REF_pave.dict
chrName.txt					 HPV18REF_pave.fa
chrStart.txt					 HPV18REF_pave.fa.amb
gencode.v43.annotation.gtf			 HPV18REF_pave.fa.ann
genecode.v43.annot.transcript.gtf		 HPV18REF_pave.fa.bwt
genome_assemblies_genome_fasta.tar		 HPV18REF_pave.fa.fai
genome_assemblies_genome_gff.tar		 HPV18REF_pave.fa.pac
GRCh37.alu.bed					 HPV18REF_pave.fa.sa
GRCh37.alue.bed					 HPV33.fa
GRCh37.ehancer.bed				 HPV33.fa.amb
GRCh37.fa					 HPV33.fa.ann
GRCh37.fa.fai					 HPV33.fa.bwt
GRCh37.repeatMasker.bed				 HPV33.fa.fai
GRCh37.USCSgene.bed				 HPV33.fa.pac
GRCh37.USCSgene.ncbi.bed			 HPV33.fa.sa
hg19.HPV16-104.fa				 HPV33REF_pave.dict
hg19.HPV16-104.fa.amb				 HPV33REF_pave.fa
hg19.HPV16-104.fa.ann				 HPV33REF_pave.fa.amb
hg19.HPV16-104.fa.bwt				 HPV33REF_pave.fa.ann
hg19.HPV16-104.fa.fai				 HPV33REF_pave.fa

In [8]:
geneDic = {}
gtfFileGenome = '/home/wenjingu/remillsscr/HPV_fusion/ref_hg19/Homo_sapiens.GRCh38.105.transcript.gtf'
with open(gtfFileGenome) as genome:
    rows = genome.read().rstrip().split('\n')
    for eachRow in rows:
        hasGeneName = False
        hasTrID = False
        for eachslot in eachRow.split('\t')[8].split(';'):
            if eachslot.split('"')[0] == ' transcript_id ':
                trID = eachslot.split('"')[1]
                hasTrID = True
            if eachslot.split('"')[0] == ' gene_name ':
                geneName = eachslot.split('"')[1]
                hasGeneName = True
        if not hasTrID:
            print(eachRow)
        if not hasGeneName:
            geneName = trID
        chro = eachRow.split('\t')[0]
        start = eachRow.split('\t')[3]
        end = eachRow.split('\t')[4]
        if trID not in geneDic:
            geneDic[trID] = [geneName,chro,start,end]
        else:
            print(eachRow)

In [9]:
def readData(path):
    samples = os.listdir(path)
    selectedNewContig = {}
    sampleWithFusions = 0
    re_run = []
    for sample in samples:
        if 'Sample_' in sample:
            resPath = f"{path}/{sample}/call_fusion_virus/VirusFusionPointContig.txt"
            resdicPath = f"{path}/{sample}/call_fusion_virus/filteredSelectedContig.pickle"
            #print(resPath)
            if not os.path.isfile(resdicPath):
                print(sample)
                re_run.append(sample)
            else:
                with open(resdicPath,'rb') as resDicFile:
                    resDic = pickle.load(resDicFile)

                    selectedNewContig[sample] = resDic
                sampleWithFusions += 1
    return selectedNewContig, re_run
    #             else:
    #                 print(sample)


In [10]:
!ls /home/wenjingu/scratch/MCV_fusion/

data	    MCC_MCPV.xlsx	     MCV.fasta	raw_fastq2
fastq_data  MCC_Sup_Table_1_QC.xlsx  raw_fastq	searcHPV_result


In [11]:
path1 = '/home/wenjingu/scratch/MCV_fusion/searcHPV_result/'
all_dic,re_run = readData(path1)

In [12]:
all_dic

{'Sample_83092': {'7.117090192': [['Contig1',
    [[127, '127M85S', 2214, [(0, 127), (4, 85)]]],
    [[87, '123S87M2S', 117090191, [(4, 123), (0, 87), (4, 2)]]],
    212,
    6,
    2341,
    'lowConfidence']],
  '1.179108371': [['Contig1',
    [[36, '36M249S', 4382, [(0, 36), (4, 249)]]],
    [[256, '256M29S', 179108431, [(0, 256), (4, 29)]]],
    285,
    3,
    4418,
    'lowConfidence']],
  '17.47747185': [['Contig1',
    [[171, '171M93S', 2027, [(0, 171), (4, 93)]]],
    [[82, '170S82M12S', 47747184, [(4, 170), (0, 82), (4, 12)]]],
    264,
    6,
    2198,
    'lowConfidence']],
  '5.77958622': [['Contig1',
    [[179, '70S179M', 2478, [(4, 70), (0, 179)]]],
    [[74, '175S74M', 77958621, [(4, 175), (0, 74)]]],
    249,
    3,
    2478,
    'lowConfidence']],
  'X.85802448': [['Contig1',
    [[213, '241S213M', 4306, [(4, 241), (0, 213)]]],
    [[240, '240M214S', 85802207, [(0, 240), (4, 214)]]],
    454,
    15,
    4306]],
  '3.93470523': [['Contig29',
    [[85, '398H85M337H', 31

In [13]:
geneDic

{'ENST00000379236': ['TNFRSF4', '1', '1211340', '1214153'],
 'ENST00000497869': ['TNFRSF4', '1', '1211340', '1214138'],
 'ENST00000453580': ['TNFRSF4', '1', '1212019', '1213498'],
 'ENST00000328596': ['TNFRSF18', '1', '1203508', '1206571'],
 'ENST00000379268': ['TNFRSF18', '1', '1203508', '1206592'],
 'ENST00000486728': ['TNFRSF18', '1', '1203844', '1205680'],
 'ENST00000379265': ['TNFRSF18', '1', '1203844', '1206571'],
 'ENST00000673477': ['ATAD3B', '1', '1471765', '1497848'],
 'ENST00000472194': ['ATAD3B', '1', '1478026', '1497848'],
 'ENST00000378736': ['ATAD3B', '1', '1479049', '1482662'],
 'ENST00000485748': ['ATAD3B', '1', '1483485', '1496202'],
 'ENST00000474481': ['ATAD3B', '1', '1484569', '1496201'],
 'ENST00000308647': ['ATAD3B', '1', '1471784', '1496201'],
 'ENST00000565563': ['ENST00000565563', '1', '1249777', '1251334'],
 'ENST00000442483': ['ENST00000442483', '1', '2212523', '2220738'],
 'ENST00000416931': ['MTND1P23', '1', '629062', '629433'],
 'ENST00000428803': ['RPL7P

In [14]:
MCVinfo

,Sample_ID,Sample name,MCPV reads,Total reads,Unnamed: 4,Unnamed: 5
0,4311-T,103105,0.0,8.373840e+06,0.000000,1.000000
1,4518T,126491,32.0,1.342001e+07,0.002384,1.002384
2,4671T,126521,56.0,9.628085e+06,0.005816,1.005816
3,HNSpore4125T,83114,118.0,1.562227e+07,0.007553,1.007553
4,4713T_MiOto_MCC,152-CB-70,100.0,1.091653e+07,0.009160,1.009160
5,4490T,126469,160.0,1.294672e+07,0.012358,1.012358
6,4199-T,92436,217.0,1.714999e+07,0.012653,1.012653
7,4268-T,96939,99.0,6.703120e+06,0.014769,1.014769
8,4516T,126489,292.0,1.263846e+07,0.023104,1.023104
9,4478T,126461,1207.0,1.654602e+07,0.072948,1.072948


In [15]:
all_dic.keys()

dict_keys(['Sample_83092', 'Sample_126501', 'Sample_83096', 'Sample_83084', 'Sample_126491', 'Sample_88113', 'Sample_83094', 'Sample_83088', 'Sample_92434', 'Sample_152-CB-66', 'Sample_83098', 'Sample_152-CB-70', 'Sample_83100', 'Sample_126469', 'Sample_83104', 'Sample_126495', 'Sample_83108', 'Sample_83110', 'Sample_109326', 'Sample_126499', 'Sample_83102', 'Sample_126461', 'Sample_96937', 'Sample_96931', 'Sample_126515', 'Sample_92436', 'Sample_83112', 'Sample_96935', 'Sample_83106', 'Sample_126521', 'Sample_83114', 'Sample_83116', 'Sample_83090', 'Sample_126505', 'Sample_126457', 'Sample_126489', 'Sample_126467', 'Sample_96918', 'Sample_83086', 'Sample_152-CB-60', 'Sample_126517', 'Sample_152-CB-64', 'Sample_109324', 'Sample_88107', 'Sample_96933', 'Sample_126527', 'Sample_126497', 'Sample_96939', 'Sample_152-CB-68', 'Sample_152-CB-62', 'Sample_103105', 'Sample_126523', 'Sample_126511', 'Sample_152-CB-58'])

In [16]:
#sample not in the table
missing_sample = 'missing_sample.txt'
with open(missing_sample,'w') as output:
    for sample in MCVinfo['Sample name']:
        if str(sample) != 'nan':
            if 'Sample_' + str(sample) not in all_dic:
                print(sample)
                output.write(f'{sample}\n')

In [17]:
!pwd

/nfs/turbo/remillsscr/wenjingu/MCV_fusion


In [18]:
!head missing_sample.txt

In [19]:
#add integrations
integrations = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        insList = []
        for each in all_dic['Sample_' + str(sample)]:
            chro = each.split('.')[0]
            site = each.split('.')[1]
            ins = 'chr' + chro + ':' + site
            insList.append(ins)
        integrations.append(';'.join(insList))
    else:
        integrations.append('')

In [20]:
#add number of integrations, don't add for RM samples
numIntegrations = []
i = 0
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        numIntegrations.append(len(all_dic['Sample_' + str(sample)].keys()))
    else:
        numIntegrations.append('NaN')


In [21]:
MCVinfo['Num MCV Integration'] = numIntegrations

In [22]:
numIntegrations

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 6,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 0,
 1,
 5,
 3,
 3,
 3,
 3,
 5,
 3,
 2,
 1,
 0,
 2,
 3,
 2,
 2,
 3,
 0,
 7,
 9,
 30,
 'NaN',
 'NaN',
 'NaN',
 'NaN']

In [23]:
integrations

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'chr5:2741236',
 'chr12:69563890;chr12:69757363',
 '',
 'chr6:10418728',
 '',
 '',
 'chr4:105235259;chr1:179121589',
 '',
 'chr14:98704650',
 'chr13:99201258',
 'chr11:79599498',
 'chr15:90814342',
 'chr11:35014065;chr11:35013925',
 'chr9:4483660;chr19:2221446;chr11:65303772;chr19:2082235;chr11:65417650;chr12:123401823',
 'chr13:55040642;chr13:55054381',
 'chr6:55402204',
 'chr7:83687568',
 'chr14:35350078;chr5:68210749',
 '',
 'chr6:2242232;chr6:1873753',
 'chr4:3854511;chr4:3716855',
 'chr19:843984',
 'chr8:113149369',
 '',
 'chr1:10676879',
 'chr3:19439911;chr3:160301146;chr6:140471786;chr7:146169649;chr3:160278945',
 'chr13:71442577;chr5:65657631;chr13:71555385',
 'chr14:61740949;chr19:13088953;chr19:12683207',
 'chr12:123453870;chr12:123118669;chr12:123453977',
 'chr1:151449050;chr10:66567821;chr1:151583668',
 'chr11:91265369;chr7:123441146;chr6:20700842;chr6:20703477;chr7:123727734',
 'chrX:118793564;chr10:61891599;chr10:61892816

In [24]:
MCVinfo['MCV Integration'] = integrations

In [25]:
#add MCV breakpoints
mcvBreakpoints = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        insListForSample = []
        for each in all_dic['Sample_' + str(sample)]:
            insList = []
            for contig in all_dic['Sample_' + str(sample)][each]:
                if contig[-1] != 'lowConfidence':
                    mcvIns = str(contig[-1])
                else:
                    mcvIns = str(contig[-2])
                insList.append(mcvIns)
            insListForSample.append(','.join(insList))
        mcvBreakpoints.append(';'.join(insListForSample))
    else:
        mcvBreakpoints.append('')



In [26]:
mcvBreakpoints

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '435,4101',
 '762;2413',
 '',
 '3051,1938',
 '',
 '',
 '1200;4065',
 '',
 '1822,2752',
 '2518,1884',
 '2922,2047',
 '3246,3731',
 '2664,2644;2664,2644',
 '4608;607;2269;1095;2269;2463',
 '1279,3637;5174',
 '3687,5385',
 '2982,4089',
 '4676;4010',
 '',
 '3875;3874',
 '4382;4763',
 '285',
 '4426',
 '',
 '223,389',
 '355;2538;2480;4685;5117',
 '4241;4241;4516',
 '4888;4771;1339',
 '1776;2295;1776',
 '3619;1551,3647;1454',
 '4377;4342;2341;4444;1348',
 '2103;4894;4895',
 '4905;1763',
 '802,2491',
 '',
 '3338;727',
 '4174;2556;1984',
 '439;663',
 '3351;2500',
 '4314;305;1520',
 '',
 '2786,3992;2808;5077;4840;1850;2786,3992;349',
 '1867;4833;685;2196;4922;4389;351;3260;377',
 '2341;4418;2198;2478;4306;3197,882;3680;4123;2791;694;2310;3670;183;1068;1198;4714;4540,882;2402;1884;2736;4985,882;1818;3197,882;694;557,1719;2296;2814,362;184;183;2237',
 '',
 '',
 '',
 '']

In [27]:
MCVinfo['MCV integration breakpoints on MCV genome'] = mcvBreakpoints

In [28]:
#Total # Host Genes with integration

In [29]:
geneDic = {}
gtfFileGenome = '/home/wenjingu/remillsscr/HPV_fusion/ref_hg19/Homo_sapiens.GRCh38.105.transcript.gtf'
with open(gtfFileGenome) as genome:
    rows = genome.read().rstrip().split('\n')
    for eachRow in rows:
        hasGeneName = False
        for each in eachRow.split('\t')[8].split(';'):
            if each.split('"')[0] == ' gene_name ':
                geneName = each.split('"')[1]
                hasGeneName = True
            if each.split('"')[0] == ' transcript_id ':
                trID = each.split('"')[1]
        if not hasGeneName:
            geneName = trID
        chro = eachRow.split('\t')[0]
        start = eachRow.split('\t')[3]
        end = eachRow.split('\t')[4]
        if trID not in geneDic:
            geneDic[trID] = [geneName,chro,start,end]
        else:
            print(eachRow)

In [30]:
geneNum = []
geneListCol = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        geneListSample = []
        geneNumSample = []
        for each in all_dic['Sample_' + str(sample)]:
            geneList = []
            chro = each.split('.')[0]
            site = each.split('.')[1]
            for gene in geneDic:
                if chro == geneDic[gene][1] and int(site)>= int(geneDic[gene][2]) and int(site) <= int(geneDic[gene][3]):
                    geneList.append(geneDic[gene][0])
            geneNumSample += geneList
            geneList = set(geneList)
            geneListSample.append(geneList)
    geneNum.append(len(set(geneNumSample)))
    geneListCol.append(geneListSample)
        

In [31]:
geneListColNew = []
for each in geneListCol:
    insGeneList = []
    for ins in each:
        insGeneList.append(','.join(ins))
    geneListColNew.append(';'.join(insGeneList))

In [32]:
MCVinfo['Total # Host Genes with integration'] = geneNum

In [33]:
MCVinfo['Host Genes with integration'] = geneListColNew

In [34]:
#Integrated in at least 1 human gene
inGeneList = []
for each in geneNum:
    inGene = False
    if each > 0:
        inGene = True
    inGeneList.append(inGene)

In [35]:
MCVinfo['Integrated in at least 1 human gene'] = inGeneList

In [36]:
#Integrated in at least 1 human cancer gene (COSMIC Teir1/2)
cancerGene = '/nfs/turbo/remillsscr/wenjingu/HPV_fusion/targeted_exom/our_pip/newVersion/analysis/uniprot_COSMIC_oncogenes_list.xlsx'
cancerGeneDf = pd.read_excel(cancerGene)


In [37]:
cancerGeneDf[(cancerGeneDf['Tier'] == 1) | (cancerGeneDf['Tier'] == 2)]

,Gene Symbol,Entrez GeneId,Genome Location,Tier,Role in Cancer
0,A1CF,29974.0,10:50799421-50885675,2,oncogene
1,ABI1,10006.0,10:26746593-26860935,1,"TSG, fusion"
2,ABL1,25.0,9:130713946-130885683,1,"oncogene, fusion"
3,ABL2,27.0,1:179099327-179229601,1,"oncogene, fusion"
4,ACKR3,57007.0,2:236569641-236582358,1,"oncogene, fusion"
...,...,...,...,...,...
728,ZNF429,353088.0,19:21505564-21538078,2,NaN
729,ZNF479,90827.0,7:57119614-57139864,2,NaN
730,ZNF521,25925.0,18:25061926-25352152,1,"oncogene, fusion"
731,ZNRF3,84133.0,22:28883592-29057487,2,TSG


In [38]:
cancerGeneList = []
cancerGeneDetail = []
for sample in MCVinfo['Sample name']:
    cancerGene = False
    sampleName = 'Sample_' + str(sample)
    cancerGeneNameSample = []
    if sampleName in all_dic:
        for each in all_dic[sampleName]:
            cancerGeneName = []
            chro = each.split('.')[0]
            site = each.split('.')[1]
            i = 0
            for gene in cancerGeneDf['Gene Symbol']:
                if cancerGeneDf['Tier'][i] ==1  or cancerGeneDf['Tier'][i] == 2:
                    chro2 = cancerGeneDf['Genome Location'][i].split(":")[0]
                    start = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[0]
                    end = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[1]
                    #print(chro2,start,end,cancerGeneDf['Genome Location'][i],gene)
                    if chro2!='' and start != '' and end != '':
                        if chro == chro2 and int(site) >= int(start) and int(site) <= int(end):
                            #print(cancerGene)
                            cancerGene = True
                            cancerGeneName.append(cancerGeneDf['Gene Symbol'][i])
                i += 1
            print(set(cancerGeneName))
            cancerGeneNameSample.append(",".join(set(cancerGeneName)))

    print(cancerGeneNameSample)
    cancerGeneDetail.append(';'.join(cancerGeneNameSample))                
    cancerGeneList.append(cancerGene)




[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
set()
['']
set()
set()
['', '']
[]
set()
['']
[]
[]
{'TET2'}
{'ABL2'}
['TET2', 'ABL2']
[]
set()
['']
set()
['']
set()
['']
{'BLM'}
['BLM']
set()
set()
['', '']
set()
set()
set()
set()
set()
set()
['', '', '', '', '', '']
set()
set()
['', '']
set()
['']
set()
['']
set()
set()
['', '']
[]
set()
set()
['', '']
set()
set()
['', '']
set()
['']
{'CSMD3'}
['CSMD3']
[]
set()
['']
set()
set()
set()
{'CNTNAP2'}
set()
['', '', '', 'CNTNAP2', '']
set()
set()
set()
['', '', '']
{'HIF1A'}
set()
set()
['HIF1A', '', '']
set()
set()
set()
['', '', '']
set()
set()
set()
['', '', '']
set()
set()
set()
set()
set()
['', '', '', '', '']
set()
set()
set()
['', '', '']
set()
set()
['', '']
set()
['']
[]
set()
set()
['', '']
set()
set()
set()
['', '', '']
set()
set()
['', '']
set()
set()
['', '']
set()
set()
set()
['', '', '']
[]
set()
set()
set()
set()
set()
set()
set()
['', '', '', '', '', '', '']
set()
set()
set()
set()
set()
set()
set()
set()
set()
['', '', '', '', '', '', '',

In [39]:
cancerGeneList = []
cancerGeneDetail = []
for sample in MCVinfo['Sample name']:
    cancerGene = False
    sampleName = 'Sample_' + str(sample)
    cancerGeneNameSample = []
    if sampleName in all_dic:
        for each in all_dic[sampleName]:
            chro = each.split('.')[0]
            site = each.split('.')[1]
            i = 0
            for gene in cancerGeneDf['Gene Symbol']:
                if cancerGeneDf['Tier'][i] ==1  or cancerGeneDf['Tier'][i] == 2:
                    chro2 = cancerGeneDf['Genome Location'][i].split(":")[0]
                    start = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[0]
                    end = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[1]
                    #print(chro2,start,end,cancerGeneDf['Genome Location'][i],gene)
                    if chro2!='' and start != '' and end != '':
                        if chro == chro2 and int(site) >= int(start) and int(site) <= int(end):
                            #print(cancerGene)
                            cancerGene = True
                            cancerGeneNameSample.append(cancerGeneDf['Gene Symbol'][i])
                i += 1
            
    cancerGeneDetail.append(','.join(set(cancerGeneNameSample)))                
    cancerGeneList.append(cancerGene)


In [40]:
cancerGeneDetail

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'TET2,ABL2',
 '',
 '',
 '',
 '',
 'BLM',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'CSMD3',
 '',
 '',
 'CNTNAP2',
 '',
 'HIF1A',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'KMT2A,ABL2,TSC1,MYCL,KAT6A,PTPRK',
 '',
 '',
 '',
 '']

In [41]:
MCVinfo['Integrated in at least 1 human cancer gene (COSMIC Teir1/2)'] = cancerGeneList

In [42]:
MCVinfo['Integrated in at least 1 human cancer gene (COSMIC Teir1/2); Gene Name'] = cancerGeneDetail

In [43]:
MCVinfo

,Sample_ID,Sample name,MCPV reads,Total reads,Unnamed: 4,Unnamed: 5,Num MCV Integration,MCV Integration,MCV integration breakpoints on MCV genome,Total # Host Genes with integration,Host Genes with integration,Integrated in at least 1 human gene,Integrated in at least 1 human cancer gene (COSMIC Teir1/2),Integrated in at least 1 human cancer gene (COSMIC Teir1/2); Gene Name
0,4311-T,103105,0.0,8.373840e+06,0.000000,1.000000,0,,,0,,False,False,
1,4518T,126491,32.0,1.342001e+07,0.002384,1.002384,0,,,0,,False,False,
2,4671T,126521,56.0,9.628085e+06,0.005816,1.005816,0,,,0,,False,False,
3,HNSpore4125T,83114,118.0,1.562227e+07,0.007553,1.007553,0,,,0,,False,False,
4,4713T_MiOto_MCC,152-CB-70,100.0,1.091653e+07,0.009160,1.009160,0,,,0,,False,False,
5,4490T,126469,160.0,1.294672e+07,0.012358,1.012358,0,,,0,,False,False,
6,4199-T,92436,217.0,1.714999e+07,0.012653,1.012653,0,,,0,,False,False,
7,4268-T,96939,99.0,6.703120e+06,0.014769,1.014769,0,,,0,,False,False,
8,4516T,126489,292.0,1.263846e+07,0.023104,1.023104,0,,,0,,False,False,
9,4478T,126461,1207.0,1.654602e+07,0.072948,1.072948,0,,,0,,False,False,


In [44]:
#annotate MCV genes

In [45]:
MCV_dic = {'VP2':[[465,1190]],'VP1':[[1156,2427]],'large_T':[[2503,4722],[5154,5387]],'small_T':[[4827,5387]]}

In [46]:
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        MCV_sample_list = []
        for ins in all_dic['Sample_' + str(sample)]:
            MCV_ins_list = []
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_site_list.append(MCV_gene)
                            has_gene = True
                if not has_gene:
                    MCV_site_list.append('')
                MCV_ins_list.append('+'.join(MCV_site_list))
            MCV_sample_list.append(','.join(MCV_ins_list))
        MCV_gene_list.append(';'.join(MCV_sample_list))


In [47]:
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    MCV_sample_list = []
    if str(sample) != 'nan':
        
        for ins in all_dic['Sample_' + str(sample)]:
            
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_sample_list.append(MCV_gene)
                            has_gene = True
       
               
    MCV_gene_list.append(','.join(set(MCV_sample_list)))
    


In [48]:
MCVinfo['MCV breakpoints fell into MCV gene'] = MCV_gene_list 

In [49]:
MCVinfo

,Sample_ID,Sample name,MCPV reads,Total reads,Unnamed: 4,Unnamed: 5,Num MCV Integration,MCV Integration,MCV integration breakpoints on MCV genome,Total # Host Genes with integration,Host Genes with integration,Integrated in at least 1 human gene,Integrated in at least 1 human cancer gene (COSMIC Teir1/2),Integrated in at least 1 human cancer gene (COSMIC Teir1/2); Gene Name,MCV breakpoints fell into MCV gene
0,4311-T,103105,0.0,8.373840e+06,0.000000,1.000000,0,,,0,,False,False,,
1,4518T,126491,32.0,1.342001e+07,0.002384,1.002384,0,,,0,,False,False,,
2,4671T,126521,56.0,9.628085e+06,0.005816,1.005816,0,,,0,,False,False,,
3,HNSpore4125T,83114,118.0,1.562227e+07,0.007553,1.007553,0,,,0,,False,False,,
4,4713T_MiOto_MCC,152-CB-70,100.0,1.091653e+07,0.009160,1.009160,0,,,0,,False,False,,
5,4490T,126469,160.0,1.294672e+07,0.012358,1.012358,0,,,0,,False,False,,
6,4199-T,92436,217.0,1.714999e+07,0.012653,1.012653,0,,,0,,False,False,,
7,4268-T,96939,99.0,6.703120e+06,0.014769,1.014769,0,,,0,,False,False,,
8,4516T,126489,292.0,1.263846e+07,0.023104,1.023104,0,,,0,,False,False,,
9,4478T,126461,1207.0,1.654602e+07,0.072948,1.072948,0,,,0,,False,False,,


In [50]:
MCVinfo.to_excel('MCV_info_v2.xlsx')

In [51]:
#link plot

In [52]:
hpvIntegrationBed = "MCVIntegration.bed"
genomeIntegrationBed = "genomeIntegration_MCV.bed"
with open(genomeIntegrationBed,'w') as outputGenome:
    with open(hpvIntegrationBed,'w') as outputHPV:
        for sample in MCVinfo['Sample name']:
            MCV_sample_list = []
            if str(sample) != 'nan':

                for ins in all_dic['Sample_' + str(sample)]:
                    chro = ins.split('.')[0]
                    site = ins.split('.')[1]
                    for contig in all_dic['Sample_' + str(sample)][ins]:
                        if contig[-1] != 'lowConfidence':
                            MCV_site = contig[-1]
                        else:
                            MCV_site = contig[-2]
                        has_gene = False
                        for MCV_gene in MCV_dic:
                            k = 1
                            for slot in MCV_dic[MCV_gene]:
                                if MCV_site >= slot[0] and MCV_site <= slot[1]:
                                    has_gene = True
                                    if len(MCV_dic[MCV_gene]) > 1:
                                        outputHPV.write(MCV_gene + f'_{str(k)}' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n')
                                    else:
                                        outputHPV.write(MCV_gene + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n')
                                    outputGenome.write('chr' + chro + '\t' + site + '\t' + site + '\n')
                                k += 1
                        if not has_gene:
                            if MCV_site >= 0 and MCV_site <= 465:
                                outputHPV.write('intergenic_1' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            elif MCV_site >= 4722 and MCV_site <= 4827:
                                outputHPV.write('intergenic_3' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            elif MCV_site >= 2427 and MCV_site <= 2503: 
                                outputHPV.write('intergenic_2' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            else:
                                print(MCV_site)
                            outputGenome.write('chr' + chro + '\t' + site + '\t' + site + '\n')


In [53]:
MCV_dic

{'VP2': [[465, 1190]],
 'VP1': [[1156, 2427]],
 'large_T': [[2503, 4722], [5154, 5387]],
 'small_T': [[4827, 5387]]}

In [54]:
!head MCVIntegration.bed

intergenic_1	17400000	17400000
large_T_1	164040000	164040000
VP2	30480000	30480000
VP1	96520000	96520000
large_T_1	122040000	122040000
VP1	77520000	77520000
VP1	48000000	48000000
large_T_1	162600000	162600000
VP1	72880000	72880000
large_T_1	110080000	110080000


In [55]:
!head genomeIntegration_MCV.bed

chr5	2741236	2741236
chr5	2741236	2741236
chr12	69563890	69563890
chr12	69757363	69757363
chr6	10418728	10418728
chr6	10418728	10418728
chr4	105235259	105235259
chr1	179121589	179121589
chr14	98704650	98704650
chr14	98704650	98704650


In [56]:
#observed MCV gene count
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        
        for ins in all_dic['Sample_' + str(sample)]:
            
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_gene_list.append(MCV_gene)
                            has_gene = True
                
       
               
    
    
    
print(MCV_gene_list)
countDic = dict(Counter(MCV_gene_list))
countDfMCV = pd.DataFrame.from_dict(countDic,orient='index')

['large_T', 'VP2', 'VP1', 'large_T', 'VP1', 'VP1', 'large_T', 'VP1', 'large_T', 'large_T', 'VP1', 'large_T', 'VP1', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'VP2', 'VP1', 'VP2', 'VP1', 'VP1', 'large_T', 'large_T', 'small_T', 'large_T', 'large_T', 'small_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'large_T', 'small_T', 'large_T', 'large_T', 'large_T', 'small_T', 'VP1', 'VP1', 'VP1', 'VP1', 'large_T', 'VP1', 'large_T', 'VP1', 'large_T', 'large_T', 'VP1', 'large_T', 'VP1', 'VP1', 'small_T', 'small_T', 'small_T', 'VP1', 'VP2', 'large_T', 'VP2', 'large_T', 'large_T', 'VP1', 'VP2', 'large_T', 'large_T', 'VP1', 'large_T', 'large_T', 'large_T', 'small_T', 'small_T', 'VP1', 'large_T', 'large_T', 'VP1', 'small_T', 'VP2', 'VP1', 'small_T', 'large_T', 'large_T', 'VP1', 'large_T', 'VP1', 'large_T', 'large_T', 'VP2', 'large_T', 'large_T', 'large_T', 'VP2', 'VP1', 'large_T', 'VP2', 'VP1', 'large_T', 'large_

In [57]:
#expect hpv integration count
totSite = 0
for each in countDic:
    totSite += countDic[each]
mcvLenDic = {}
totLen = 0
for site in MCV_dic:
    mcvLen = 0
    for each in MCV_dic[site]:
        mcvLen += int(each[1])-int(each[0])
    mcvLenDic[site] = mcvLen
    totLen += mcvLen
expectMCV = {}
for each in mcvLenDic:
    prob = mcvLenDic[each]/totLen
    expectCount = round(totSite*prob)
    expectMCV[each] = expectCount

In [58]:
expectMCVDf = pd.DataFrame.from_dict(expectMCV,orient = 'index')
countDfHPV = pd.DataFrame.merge(countDfMCV,expectMCVDf,left_index = True,right_index=True)
countDfHPV.columns = ['observed','expected']
countDfHPV.to_csv('MCV_gene.csv')

In [59]:
countDfHPV

,observed,expected
large_T,59,59
VP2,15,17
VP1,34,30
small_T,12,13


In [60]:
#microhomology

In [102]:
#get blockList
def getBlock2(bam):
    sam = pysam.AlignmentFile(bam, "rb")
    blockList = {}#blockList[contig] = [start,end,cigarString]
    high_quality_contig = []
    for read in sam.fetch():
        start = read.pos
        contig = read.qname
        blocks = read.get_blocks()
        cigar = read.cigartuples
        length = read.infer_read_length()
        if read.mapping_quality >= 0:
            #if start with clip
            high_quality_contig.append(contig)
            start = 0
            end = 0
            for eachSlot in cigar:
                end += eachSlot[1]
                if not read.is_reverse:
                    if contig not in blockList:
                        blockList[contig] = [[start,end,eachSlot[0]]]
                    else:
                        blockList[contig].append([start,end,eachSlot[0]])
                else:
                    if contig not in blockList:
                        blockList[contig] = [[length-end + 1,length-start + 1,eachSlot[0]]]
                    else:
                        blockList[contig].append([length-end + 1,length-start + 1,eachSlot[0]])
                start += eachSlot[1]
        newBlockList = {}
        for contig in blockList:
            for each in  blockList[contig]:
                if each[2] == 0:
                    if contig not in newBlockList:
                        newBlockList[contig] = [[each[0],each[1]]]
                    else:
                        newBlockList[contig].append([each[0],each[1]])
                
    return newBlockList,high_quality_contig

In [103]:
#count overlap
def countOverlap(selectedContig,genomeBlock,hpvBlock):
    scoreDic = {}
    for each in selectedContig:
        score = 0
        overlap = False
        cleanEnd = False
        for slot1 in genomeBlock[each]:
            start1 = slot1[0]
            end1 = slot1[1]
            for slot2 in hpvBlock[each]:
                start2 = slot2[0]
                end2 = slot2[1]
                # overlap
                oldScore = score
                if start2 <= end1 and start1 <= end2:
                    overlap = True
                    if end1-start2 > end2-start1:
                        newScore = end2-start1 + 1
                    else:
                        newScore = end1-start2 + 1
                    if newScore > oldScore:
                        score = newScore
        #clean end
        if overlap == False:
            score = 0
            for slot1 in genomeBlock[each]:
                start1 = slot1[0]
                end1 = slot1[1]
                for slot2 in hpvBlock[each]:
                    start2 = slot2[0]
                    end2 = slot2[1]
                    #clean end
                    if start2 == end1 + 1 or start1 == end2 + 1:
                        cleanEnd = True
                        score += 0
            #gap
            if overlap == False and cleanEnd == False:
                score = 0
                for slot1 in genomeBlock[each]:
                    start1 = slot1[0]
                    end1 = slot1[1]
                    for slot2 in hpvBlock[each]:
                        start2 = slot2[0]
                        end2 = slot2[1]
                        #gap
                        oldScore = score
                        if end1 < start2 or end2 < start1:
                            if end1 < start2:
                                newScore2 = -(start2 - end1 - 1)
                            elif end2 < start1:
                                newScore2 = -(start1 - end2 - 1)
                            if newScore2 > oldScore and oldScore != 0:
                                score = newScore2
                                #print(newScore2)
                            elif oldScore == 0:
                                score = newScore2
        #filter score > 100 or <-100
#         if score < 100 and score > -100:
        scoreDic[each] = score
    return(scoreDic)

In [104]:
scoreList = []

path = '/home/wenjingu/scratch/MCV_fusion/searcHPV_result/'
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        sampleName = 'Sample_' + str(sample)
        resPath = f'{path}/{sampleName}/call_fusion_virus/'
        finalRes = f'{resPath}/filteredSelectedContig.pickle'
        with open(finalRes,'rb') as inputFile:
            resDic = pickle.load(inputFile)
        for each in resDic:
            if os.path.isdir(resPath + each):
                alignGenome = f'{resPath}/{each}/{each}.contigToGenome.sort.bam'
                alignHPV = f'{resPath}/{each}/{each}.contigToVirus.sort.bam'
                genomeBlock,high_quality_contig = getBlock2(alignGenome)
                hpvBlock,high_quality_contig2 = getBlock2(alignHPV)
                #selected Contig
                selectedContig = []
                for eachSlot in resDic[each]:
                    selectedContig.append(eachSlot[0])
                print(high_quality_contig)
                print(selectedContig)
                selectedContig = [each for each in selectedContig if each in high_quality_contig]
                selectedContig = [each for each in selectedContig if each in high_quality_contig2]
                #count overlap bp
                print(hpvBlock)
                print(selectedContig)
                scoreDic = countOverlap(selectedContig,genomeBlock,hpvBlock)
                for contig in scoreDic:
                    scoreList.append(scoreDic[contig])
                print(sample,each,scoreDic)
        #print(scoreList)

['Contig2', 'Contig1']
['Contig2', 'Contig1']
{'Contig2': [[274, 445], [208, 269]], 'Contig1': [[198, 484]]}
['Contig2', 'Contig1']
126515 5.2741236 {'Contig2': 1, 'Contig1': 4}
['Contig1']
['Contig1']
{'Contig1': [[283, 500]]}
['Contig1']
126517 12.69563890 {'Contig1': 3}
['Contig1']
['Contig1']
{'Contig1': [[137, 407]]}
['Contig1']
126517 12.69757363 {'Contig1': 4}
['Contig2', 'Contig1']
['Contig2', 'Contig1']
{'Contig1': [[127, 192], [0, 118]], 'Contig2': [[259, 411]]}
['Contig2', 'Contig1']
126501 6.10418728 {'Contig2': 2, 'Contig1': 0}
['Contig2', 'Contig1']
['Contig1']
{'Contig1': [[0, 141]]}
['Contig1']
83108 4.105235259 {'Contig1': 6}
['Contig1']
['Contig1']
{'Contig1': [[0, 120]]}
['Contig1']
83108 1.179121589 {'Contig1': 4}
['Contig2', 'Contig1']
['Contig2', 'Contig1']
{'Contig2': [[0, 254], [249, 296]], 'Contig1': [[120, 355]]}
['Contig2', 'Contig1']
96933 14.98704650 {'Contig2': 1, 'Contig1': 3}
['Contig1', 'Contig2']
['Contig1', 'Contig2']
{'Contig2': [[597, 672], [329, 59

['Contig1']
['Contig1']
{'Contig1': [[112, 184], [184, 258]]}
['Contig1']
83092 16.24636375 {'Contig1': 1}
['Contig1']
['Contig1']
{'Contig2': [[2, 227]], 'Contig1': [[41, 228]]}
['Contig1']
83092 8.41942867 {'Contig1': 8}
['Contig1']
['Contig1']
{'Contig1': [[0, 146]]}
['Contig1']
83092 9.132896578 {'Contig1': 4}
['Contig1']
['Contig1']
{'Contig1': [[121, 241]]}
['Contig1']
83092 15.68090844 {'Contig1': 1}
['Contig6', 'Contig27', 'Contig14', 'Contig1', 'Contig19', 'Contig5', 'Contig2', 'Contig9', 'Contig24', 'Contig22', 'Contig3', 'Contig20', 'Contig8', 'Contig12', 'Contig14', 'Contig9', 'Contig10', 'Contig22', 'Contig28', 'Contig21', 'Contig25', 'Contig12', 'Contig19', 'Contig23', 'Contig13', 'Contig10', 'Contig14', 'Contig4', 'Contig6', 'Contig7']
['Contig19', 'Contig25']
{'Contig1': [[41, 191], [191, 242]], 'Contig6': [[94, 136], [552, 680], [136, 251]], 'Contig22': [[158, 318]], 'Contig19': [[239, 249], [1, 238], [761, 921], [430, 612]], 'Contig8': [[1, 119], [361, 442]], 'Contig2

In [105]:
!ls /home/wenjingu/scratch/MCV_fusion/searcHPV_result/

103105	126523	   83102  Sample_103105  Sample_126523	   Sample_83102
109324	126527	   83104  Sample_109324  Sample_126527	   Sample_83104
109326	152-CB-58  83106  Sample_109326  Sample_152-CB-58  Sample_83106
126457	152-CB-60  83108  Sample_126457  Sample_152-CB-60  Sample_83108
126461	152-CB-62  83110  Sample_126461  Sample_152-CB-62  Sample_83110
126467	152-CB-64  83112  Sample_126467  Sample_152-CB-64  Sample_83112
126469	152-CB-66  83114  Sample_126469  Sample_152-CB-66  Sample_83114
126489	152-CB-68  83116  Sample_126489  Sample_152-CB-68  Sample_83116
126491	152-CB-70  88107  Sample_126491  Sample_152-CB-70  Sample_88107
126495	83084	   88113  Sample_126495  Sample_83084	   Sample_88113
126497	83086	   92434  Sample_126497  Sample_83086	   Sample_92434
126499	83088	   92436  Sample_126499  Sample_83088	   Sample_92436
126501	83090	   96918  Sample_126501  Sample_83090	   Sample_96918
126505	83092	   96931  Sample_126505  Sample_83092	   Sample_96931
126511	83094	   96933  Sample_

In [106]:
print(scoreList)

[1, 4, 3, 4, 2, 0, 6, 4, 1, 3, 3, -16, 4, -1, 1, 5, 3, 9, 3, 9, 4, 6, 3, 2, 3, -4, -2, -18, 1, 2, 4, 0, 3, 1, 5, 3, 6, 4, -6, 3, 1, 4, -24, 4, 4, 13, 12, 3, 2, 2, 1, 11, -6, -15, 0, 5, 0, 4, 1, 3, 4, 3, -14, 2, 4, 0, 2, 5, 3, 3, -1, 2, 5, 4, 4, 4, -1, 4, 2, -5, 7, 3, 2, 1, 4, 8, 2, 6, 11, 1, 2, 8, 2, 16, 9, 16, 0, 10, 9, 13, 5, 5, 9, 5, 7, 2, 6, 0, 5, 0, 2, 7, 7, 1, 14, 2, 1, 8, 4, 1, 3, 0, 8, 1, 8, 5, 0, 5, 5, 0, 1, -15, 2, 11, 4, -38, -4, 1, 2]


In [65]:
expectMCV

{'VP2': 17, 'VP1': 30, 'large_T': 59, 'small_T': 13}

In [66]:
countDfMCV 

,0
large_T,59
VP2,15
VP1,34
small_T,12


In [67]:
sample = '/home/wenjingu//scratch/MCV_fusion/searcHPV_result/83092/alignment/alignment.RG.indelre.mkdup.sort.bam'
test = pysam.AlignmentFile(sample, "rb")

In [68]:
test.references

('1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'MT',
 'X',
 'Y',
 'KI270728.1',
 'KI270727.1',
 'KI270442.1',
 'KI270729.1',
 'GL000225.1',
 'KI270743.1',
 'GL000008.2',
 'GL000009.2',
 'KI270747.1',
 'KI270722.1',
 'GL000194.1',
 'KI270742.1',
 'GL000205.2',
 'GL000195.1',
 'KI270736.1',
 'KI270733.1',
 'GL000224.1',
 'GL000219.1',
 'KI270719.1',
 'GL000216.2',
 'KI270712.1',
 'KI270706.1',
 'KI270725.1',
 'KI270744.1',
 'KI270734.1',
 'GL000213.1',
 'GL000220.1',
 'KI270715.1',
 'GL000218.1',
 'KI270749.1',
 'KI270741.1',
 'GL000221.1',
 'KI270716.1',
 'KI270731.1',
 'KI270751.1',
 'KI270750.1',
 'KI270519.1',
 'GL000214.1',
 'KI270708.1',
 'KI270730.1',
 'KI270438.1',
 'KI270737.1',
 'KI270721.1',
 'KI270738.1',
 'KI270748.1',
 'KI270435.1',
 'GL000208.1',
 'KI270538.1',
 'KI270756.1',
 'KI270739.1',
 'KI270757.1',
 'KI270709.1',
 'KI270746.1',
 'KI270753.1',
 'KI270589.1',
 'KI270726.

In [69]:
test.lengths

(248956422,
 133797422,
 135086622,
 133275309,
 114364328,
 107043718,
 101991189,
 90338345,
 83257441,
 80373285,
 58617616,
 242193529,
 64444167,
 46709983,
 50818468,
 198295559,
 190214555,
 181538259,
 170805979,
 159345973,
 145138636,
 138394717,
 16569,
 156040895,
 57227415,
 1872759,
 448248,
 392061,
 280839,
 211173,
 210658,
 209709,
 201709,
 198735,
 194050,
 191469,
 186739,
 185591,
 182896,
 181920,
 179772,
 179693,
 179198,
 176845,
 176608,
 176043,
 175055,
 172810,
 168472,
 165050,
 164239,
 161802,
 161471,
 161147,
 158759,
 157432,
 155397,
 153799,
 150754,
 150742,
 148850,
 138126,
 137718,
 127682,
 112551,
 112505,
 103838,
 100316,
 99375,
 93321,
 92983,
 92689,
 91309,
 79590,
 73985,
 71251,
 66860,
 66486,
 62944,
 44474,
 43739,
 42811,
 42210,
 41891,
 41717,
 41543,
 40745,
 40191,
 40176,
 40062,
 39555,
 39050,
 38115,
 38054,
 37690,
 37240,
 36723,
 32032,
 31033,
 27745,
 22689,
 21476,
 15008,
 12399,
 8320,
 8127,
 7992,
 7642,
 7046,
 

In [70]:
chr_len = {}
for i in range(0,22):
    chr_len[int(test.references[i])] = test.lengths[i]

In [71]:
ordered_chr_len = {}
for key in sorted(chr_len):
    ordered_chr_len[key] = chr_len[key]

In [72]:
ordered_chr_len

{1: 248956422,
 2: 242193529,
 3: 198295559,
 4: 190214555,
 5: 181538259,
 6: 170805979,
 7: 159345973,
 8: 145138636,
 9: 138394717,
 10: 133797422,
 11: 135086622,
 12: 133275309,
 13: 114364328,
 14: 107043718,
 15: 101991189,
 16: 90338345,
 17: 83257441,
 18: 80373285,
 19: 58617616,
 20: 64444167,
 21: 46709983,
 22: 50818468}

In [73]:
ordered_chr_len[test.references[23]] = test.lengths[23]
ordered_chr_len[test.references[24]] = test.lengths[24]

In [74]:
ordered_chr_len

{1: 248956422,
 2: 242193529,
 3: 198295559,
 4: 190214555,
 5: 181538259,
 6: 170805979,
 7: 159345973,
 8: 145138636,
 9: 138394717,
 10: 133797422,
 11: 135086622,
 12: 133275309,
 13: 114364328,
 14: 107043718,
 15: 101991189,
 16: 90338345,
 17: 83257441,
 18: 80373285,
 19: 58617616,
 20: 64444167,
 21: 46709983,
 22: 50818468,
 'X': 156040895,
 'Y': 57227415}

In [75]:
#generate bed file
bedFile = 'MCV_track.bed'
with open(bedFile,'w') as output:
    for gene in MCV_dic:
        k = 1
        for slot in MCV_dic[gene]:
            start = str((slot[0]-1)*40000)
            end = str((slot[1]-1)*40000)
            
            if len(MCV_dic[gene]) > 1:
                output.write(f'{gene}_{str(k)}\t{start}\t{end}\n')
            else:
                output.write(f'{gene}\t{start}\t{end}\n')
            k += 1
    inter1_end = str(464*40000)
    inter2_start = str(2426*40000)
    inter2_end = str(2502*40000)
    inter3_start = str(4721*40000)
    inter3_end = str(4826*40000)
    
    output.write(f'intergenic_1\t0\t{inter1_end}\n')
    output.write(f'intergenic_2\t{inter2_start}\t{inter2_end}\n')
    output.write(f'intergenic_3\t{inter3_start}\t{inter3_end}\n')

    for chro in ordered_chr_len:
        end = str(ordered_chr_len[chro])
        output.write(f'chr{chro}\t0\t{end}\n')


In [76]:
!head MCV_track.bed

VP2	18560000	47560000
VP1	46200000	97040000
large_T_1	100080000	188840000
large_T_2	206120000	215440000
small_T	193040000	215440000
intergenic_1	0	18560000
intergenic_2	97040000	100080000
intergenic_3	188840000	193040000
chr1	0	248956422
chr2	0	242193529


In [77]:
MCV_dic

{'VP2': [[465, 1190]],
 'VP1': [[1156, 2427]],
 'large_T': [[2503, 4722], [5154, 5387]],
 'small_T': [[4827, 5387]]}

In [78]:
#generate label file
labelFile = 'label.bed'
with open(labelFile,'w') as output:
    for gene in MCV_dic:
        k = 1
        for slot in MCV_dic[gene]:
            start = str((slot[0]-1)*40000/2)
            end = str((slot[1]-1)*40000/2)
            
            if len(MCV_dic[gene]) > 1:
                output.write(f'{gene}_{str(k)}\t{start}\t{end}\t{gene}\n')
            else:
                output.write(f'{gene}\t{start}\t{end}\t{gene}\n')
            k += 1
    inter1_end = str(464*40000/2)
    inter2_start = str(2426*40000/2)
    inter2_end = str(2502*40000/2)
    inter3_start = str(4721*40000/2)
    inter3_end = str(4826*40000/2)
    output.write(f'intergenic_1\t0\t{inter1_end}\tintergenic\n')
    output.write(f'intergenic_2\t{inter2_start}\t{inter2_end}\tintergenic\n')
    output.write(f'intergenic_3\t{inter3_start}\t{inter3_end}\n')

    for chro in ordered_chr_len:
        end = str(ordered_chr_len[chro]/2)
        output.write(f'chr{chro}\t0\t{end}\tchr{chro}\n')


In [79]:
!head MCV_track.bed

VP2	18560000	47560000
VP1	46200000	97040000
large_T_1	100080000	188840000
large_T_2	206120000	215440000
small_T	193040000	215440000
intergenic_1	0	18560000
intergenic_2	97040000	100080000
intergenic_3	188840000	193040000
chr1	0	248956422
chr2	0	242193529


In [80]:
!head label.bed

VP2	9280000.0	23780000.0	VP2
VP1	23100000.0	48520000.0	VP1
large_T_1	50040000.0	94420000.0	large_T
large_T_2	103060000.0	107720000.0	large_T
small_T	96520000.0	107720000.0	small_T
intergenic_1	0	9280000.0	intergenic
intergenic_2	48520000.0	50040000.0	intergenic
intergenic_3	94420000.0	96520000.0
chr1	0	124478211.0	chr1
chr2	0	121096764.5	chr2


In [81]:
5387

5387

In [82]:
MCV_dic

{'VP2': [[465, 1190]],
 'VP1': [[1156, 2427]],
 'large_T': [[2503, 4722], [5154, 5387]],
 'small_T': [[4827, 5387]]}

In [83]:
n = 0
for read in test.fetch('10'):
    n += 1
    

In [84]:
n

0

In [85]:
!ls /home/wenjingu/scratch/MCV_fusion/raw_fastq/Sample_103105

103105_TCCGGAGA-GCCTCTAT_S10_L003_R1_001.fastq.gz
103105_TCCGGAGA-GCCTCTAT_S10_L003_R2_001.fastq.gz
